<a href="https://colab.research.google.com/github/bhanudeergasi/NullClass_Data_science_internship/blob/main/text_encoder_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Setup

In [1]:
!pip install -q transformers torch pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.2 MB/s eta 0:00:00


Imports

In [2]:
from transformers import CLIPTokenizer, CLIPTextModel, CLIPProcessor, CLIPModel
import torch
from PIL import Image

import json
import os

Mount to google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
project_path = "/content/drive/MyDrive/TASK3"

#  Load Prompts

In [5]:
with open(os.path.join(project_path, "promts.json")) as f:
    text_prompts = json.load(f)

#  Load CLIP Text Encoder

In [6]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

#  Generate Text Embeddings

In [7]:
all_embeddings = []
batch_size = 16

for i in range(0, len(text_prompts), batch_size):
    batch = text_prompts[i:i + batch_size]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = text_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    all_embeddings.append(embeddings)

final_embeddings = torch.cat(all_embeddings, dim=0)
torch.save(final_embeddings, os.path.join(project_path, "text_embeddings.pt"))
print(" Text embeddings saved to text_embeddings.pt")

 Text embeddings saved to text_embeddings.pt


In [8]:

from PIL import Image, ImageDraw
from transformers import CLIPProcessor, CLIPModel
import torch
import os

# Paths
project_path = "/content/drive/MyDrive/TASK3"
image_output_path = os.path.join(project_path, "generated_images")
os.makedirs(image_output_path, exist_ok=True)

# Generate Dummy Images
for i, prompt in enumerate(text_prompts[:10]):
    img = Image.new("RGB", (512, 512), color=(255, 255, 255))
    d = ImageDraw.Draw(img)
    d.text((10, 10), prompt[:40], fill=(0, 0, 0))  # Use part of prompt as label
    img_path = os.path.join(image_output_path, f"generated_image_{i}.png")
    img.save(img_path)

print(" Dummy images generated for testing CLIP score.")

 Dummy images generated for testing CLIP score.


#  CLIP Score Evaluation (using same prompts)

In [9]:
# Evaluate CLIP Scores
print("\n Evaluating CLIP score for image-prompt alignment...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

clip_scores = []
for i, prompt in enumerate(text_prompts[:10]):
    image_path = os.path.join(image_output_path, f"generated_image_{i}.png")
    if not os.path.exists(image_path):
        print(f" Skipping image {i} (not found)")
        continue

    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(text=[prompt], images=image, return_tensors="pt", padding=True)

    with torch.no_grad():
        outputs = clip_model(**inputs)
        score = outputs.logits_per_image.softmax(dim=1).item()
        clip_scores.append(score)
        print(f"🔍 [{i}] \"{prompt[:30]}...\" → CLIP Score: {score:.3f}")

#  Final Accuracy
passed = sum(s > 0.80 for s in clip_scores)
accuracy = passed / len(clip_scores) * 100 if clip_scores else 0
print(f"\n Final Accuracy: {accuracy:.2f}% (CLIP Score > 0.80)")



 Evaluating CLIP score for image-prompt alignment...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

🔍 [0] "A dog running on the beach..." → CLIP Score: 1.000
🔍 [1] "A futuristic cityscape at nigh..." → CLIP Score: 1.000
🔍 [2] "An astronaut riding a horse on..." → CLIP Score: 1.000
🔍 [3] "A steaming cup of coffee on a ..." → CLIP Score: 1.000
🔍 [4] "A lush forest with a waterfall..." → CLIP Score: 1.000
🔍 [5] "A spaceship landing on an alie..." → CLIP Score: 1.000
🔍 [6] "A cat sitting on a pile of boo..." → CLIP Score: 1.000
🔍 [7] "A sunset over the mountains..." → CLIP Score: 1.000
🔍 [8] "A bowl of fresh fruit on a kit..." → CLIP Score: 1.000
🔍 [9] "A robot playing the violin..." → CLIP Score: 1.000

 Final Accuracy: 100.00% (CLIP Score > 0.80)


# Final Accuracy

In [10]:
passed = sum(s > 0.80 for s in clip_scores)
accuracy = passed / len(clip_scores) * 100 if clip_scores else 0
print(f"\n Final Accuracy: {accuracy:.2f}% (CLIP Score > 0.80)")



 Final Accuracy: 100.00% (CLIP Score > 0.80)
